## Acidentes Ambientais

Import dos dados

In [ ]:
import pandas as pd 

acidentes_amb = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/acidentes_ambientais.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,22,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
acidentes_amb

,FID,id_ocorrencia,id_municipio,id_uf,id_responsavel,des_complemento_tipo_localizaca,des_complemento_tipo_evento,des_complemento_instituicao_atu,des_complemento_tipo_dano_ident,des_danos,des_ocorrencia,dt_ocorrencia,hr_ocorrencia,dt_registro,des_informacoes_adicionais,plano_emergencia,plano_emergencia_acionado,iniciados_outras_providencias,des_outras_providencias,ocorrencia_oleo,dt_primeira_obs,hr_primeira_obs,informacao_geografica,informacao_responsavel,acao_inicial_tomada,des_causa_provavel,situacao_atual_descarga,nome_comunicante,telefone_contato,des_obs,email_comunicante,nro_ocorrencia,endereco_ocorrencia,nome_instituicao_atuando,telefone_instituicao_atuando,tipo_substancia,volume_estimado,produto_nao_se_aplica,produto_perigoso,produto_nao_especificado,id_bacia_sedimentar,cpf_contato,ip_contato,legado,des_instituicao_empresa,des_funcao_comunicante,validado,tipo_comunicado,periodo_ocorrencia,periodo_primeira_obs,municipio,uf,bacia_sedimentar,origem,tipo_evento,dt_ocorrencia_feriado,dia_semana,dia_semana_primeira_obs,dia_semana_registro,institiuicoes_atuando_local,tipos_fontes_informacoes,tipos_danos_identificados,produtos_onu,produtos_outro,geom,dt_carga,feicao_proxima
0,adm_comunicacidente_p.129994b2-0030-40cf-9883-...,312,2706901.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Tombamento de caminhão carregado de biocombust...,NaN,NULL,2008-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Pilar,AL,NaN,Rodovia,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros;Polícia Rodoviária,Mídia,Óbitos/feridos,NaN,NaN,POINT (-35.9511275 -9.6016711),2021-08-07T20:46:15,NaN
1,adm_comunicacidente_p.377c6c77-2c37-400b-906b-...,313,3205309.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,Explosão em depósito de gás.,NaN,00:00,2008-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Vitória,ES,NaN,Armazenamento/depósito,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros,Mídia,Outro(s),NaN,NaN,POINT (-40.3016327 -20.2996826),2021-08-07T20:46:15,NaN
2,adm_comunicacidente_p.34f10c05-c136-4b9c-aa17-...,314,5003702.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,Vazamento de esgoto em grande quantidade causa...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Dourados,MS,NaN,Outro(s),Outro(s),NaN,NaN,NaN,NaN,Outra(s),Mídia,Fauna;Flora;População afetada/evacuada;Rio/cór...,NaN,NaN,POINT (-54.812208 -22.2206145),2021-08-07T20:46:15,NaN
3,adm_comunicacidente_p.299fd12e-ded3-43a9-9261-...,315,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Tombamento de caminhão carregado de combustíve...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,NaN,DF,NaN,Rodovia,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros;Defesa Civil,Mídia,Solo,NaN,NaN,POINT (-47.9768478 -15.82326),2021-08-07T20:46:15,NaN
4,adm_comunicacidente_p.5ae9a380-1e09-4d41-8123-...,316,4312054.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,300 Kg de produtos poluidores encontrados ás m...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Marques de Souza,RS,NaN,Outro(s),Outro(s),NaN,NaN,NaN,NaN,Outra(s),Mídia,Rio/córrego;Solo,NaN,NaN,POINT (-52.097268 -29.331113),2021-08-07T20:46:15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,adm_comunicacidente_p.54a16a16-9fe3-4a4e-bf5c-...,13940,3302403.0,19.0,9525.0,AHTS CBO TERRA BRASILIS,Após o término do abastecimento de Diesel do P...,Petróleo Brasileiro - PETROBRAS,Descarga de 200ml de óleo diesel para o mar.,NaN,NaN,2021-

Transformação da coluna "geom" em latitude e longitude.

In [ ]:
from tqdm import tqdm

acidentes_amb["lat"], acidentes_amb["lng"] = None, None

for i in tqdm(acidentes_amb["geom"].str.replace("(", "").str.replace(")", "").str.split(" ").index):
  acidentes_amb["lng"].loc[i] = float(acidentes_amb["geom"].str.replace("(", "").str.replace(")", "").str.split(" ")[i][1])
  acidentes_amb["lat"].loc[i] = float(acidentes_amb["geom"].str.replace("(", "").str.replace(")", "").str.split(" ")[i][2])


  0%|          | 0/11780 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 11780/11780 [12:48<00:00, 15.33it/s]


In [ ]:
acidentes_amb = acidentes_amb.rename(columns={"lat": "lng", "lng": "lat"})
acidentes_amb = acidentes_amb[["dt_registro", "id_municipio", "id_uf", "des_ocorrencia", "municipio", "uf", "tipos_danos_identificados", "lat", "lng"]]
acidentes_amb.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj/acidentes_amb_local.csv", index=False)

In [ ]:
acidentes_amb

,FID,id_ocorrencia,id_municipio,id_uf,id_responsavel,des_complemento_tipo_localizaca,des_complemento_tipo_evento,des_complemento_instituicao_atu,des_complemento_tipo_dano_ident,des_danos,des_ocorrencia,dt_ocorrencia,hr_ocorrencia,dt_registro,des_informacoes_adicionais,plano_emergencia,plano_emergencia_acionado,iniciados_outras_providencias,des_outras_providencias,ocorrencia_oleo,dt_primeira_obs,hr_primeira_obs,informacao_geografica,informacao_responsavel,acao_inicial_tomada,des_causa_provavel,situacao_atual_descarga,nome_comunicante,telefone_contato,des_obs,email_comunicante,nro_ocorrencia,endereco_ocorrencia,nome_instituicao_atuando,telefone_instituicao_atuando,tipo_substancia,volume_estimado,produto_nao_se_aplica,produto_perigoso,produto_nao_especificado,id_bacia_sedimentar,cpf_contato,ip_contato,legado,des_instituicao_empresa,des_funcao_comunicante,validado,tipo_comunicado,periodo_ocorrencia,periodo_primeira_obs,municipio,uf,bacia_sedimentar,origem,tipo_evento,dt_ocorrencia_feriado,dia_semana,dia_semana_primeira_obs,dia_semana_registro,institiuicoes_atuando_local,tipos_fontes_informacoes,tipos_danos_identificados,produtos_onu,produtos_outro,geom,dt_carga,feicao_proxima,lat,lng
0,adm_comunicacidente_p.129994b2-0030-40cf-9883-...,312,2706901.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Tombamento de caminhão carregado de biocombust...,NaN,NULL,2008-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Pilar,AL,NaN,Rodovia,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros;Polícia Rodoviária,Mídia,Óbitos/feridos,NaN,NaN,POINT (-35.9511275 -9.6016711),2021-08-07T20:46:15,NaN,-35.9511,-9.60167
1,adm_comunicacidente_p.377c6c77-2c37-400b-906b-...,313,3205309.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,Explosão em depósito de gás.,NaN,00:00,2008-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Vitória,ES,NaN,Armazenamento/depósito,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros,Mídia,Outro(s),NaN,NaN,POINT (-40.3016327 -20.2996826),2021-08-07T20:46:15,NaN,-40.3016,-20.2997
2,adm_comunicacidente_p.34f10c05-c136-4b9c-aa17-...,314,5003702.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,Vazamento de esgoto em grande quantidade causa...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Dourados,MS,NaN,Outro(s),Outro(s),NaN,NaN,NaN,NaN,Outra(s),Mídia,Fauna;Flora;População afetada/evacuada;Rio/cór...,NaN,NaN,POINT (-54.812208 -22.2206145),2021-08-07T20:46:15,NaN,-54.8122,-22.2206
3,adm_comunicacidente_p.299fd12e-ded3-43a9-9261-...,315,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,Tombamento de caminhão carregado de combustíve...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,NaN,DF,NaN,Rodovia,Outro(s),NaN,NaN,NaN,NaN,Corpo de Bombeiros;Defesa Civil,Mídia,Solo,NaN,NaN,POINT (-47.9768478 -15.82326),2021-08-07T20:46:15,NaN,-47.9768,-15.8233
4,adm_comunicacidente_p.5ae9a380-1e09-4d41-8123-...,316,4312054.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,300 Kg de produtos poluidores encontrados ás m...,NaN,NULL,2009-01-01 00:00:00,NaN,NaN,N,N,NaN,N,NaN,NaN,S,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN,NaN,S,NaN,M,NaN,Marques de Souza,RS,NaN,Outro(s),Outro(s),NaN,NaN,NaN,NaN,Outra(s),Mídia,Rio/córrego;Solo,NaN,NaN,POINT (-52.097268 -29.331113),2021-08-07T20:46:15,NaN,-52.0973,-29.3311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,adm_comunicacidente_p.54a16a16-9fe3-4a4e-bf5c-...,13940,3302403.0,19.0,9525.0,AHTS CBO TERRA BRASILIS,Após o término do abastecimento de

## Ocorrência de Incêndios 

Para se extrair dados de geolocalização da ocorrência de incêndios, recorreu-se à API Google Maps, que fornece o geocode a partir de apenas um endereço. Como a base só tem endereço, optou-se por essa abordagem.

In [ ]:
incendios = pd.read_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/ocorrencia_incendios.csv", sep=";")
incendios["geo_address"] = incendios["LOCALIDADE"] + " " + incendios["MUNICIPIO"] + " " + incendios["UF"]
incendios["geo_address"] = incendios["geo_address"].str.lower()
geocode_incend = pd.DataFrame(columns=["municipio", "address", "lat", "lng"])
geocode_incend["address"] = incendios["geo_address"].unique()

In [ ]:
pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.5.3-py3-none-any.whl size=38479 sha256=0fccdafd6d445e511367a79c064582b32e2560f51b2a65b25c67ae9dc1ffe751
  Stored in directory: /root/.cache/pip/wheels/fa/1a/1c/cc0b8a1652a3f06aea586b0e4714a81bafed830513969baf92
Successfully built googlemaps


O código abaixo coleta o geocode das áreas de incendios. A API KEY foi ocultada pois utiliza de conta pessoal de um dos autores do projeto. 

In [ ]:
import googlemaps
from datetime import datetime
from tqdm import tqdm

gmaps = googlemaps.Client(key="api_key")
er = []
for i in tqdm(geocode_incend.index):

  try:
    result = gmaps.geocode(geocode_incend["address"].loc[i])
    geocode_incend["lat"].loc[i] = result[0]["geometry"]["location"]["lat"]
    geocode_incend["lng"].loc[i] = result[0]["geometry"]["location"]["lng"]
  except:
    er.append(geocode_incend["address"].loc[i])

100%|██████████| 1374/1374 [14:03<00:00,  1.63it/s]


In [ ]:
geocode_incend["lat"]

,geo_address,lat,lng,LOCALIDADE,UF
0,flona de brasília brasilia df,-15.7865,-48.0578,FLONA DE BRASÍLIA,DF
1,parna e histórico do monte pascoal porto segur...,-16.8927,-39.2167,PARNA E HISTÓRICO DO MONTE PASCOAL,BA
2,apa santo antônio santa cruz cabralia ba,-16.1838,-38.9636,APA SANTO ANTÔNIO,BA
3,parque natural serra das almas rio de contas ba,-13.3716,-41.8841,PARQUE NATURAL SERRA DAS ALMAS,BA
4,vale do rio buranhém porto seguro ba,-16.4179,-39.5367,VALE DO RIO BURANHÉM,BA
...,...,...,...,...,...
1369,comunidade águas linda nova bandeirantes mt,-9.85031,-57.8153,Comunidade Águas Linda,MT
1370,penal km 60 porto velho ro,NaN,NaN,PENAL KM 60,RO
1371,faz santo onofre pocone mt,-16.2646,-56.6278,FAZ SANTO ONOFRE,MT
1372,vaquejador floriano pi,-6.80237,-43.0505,Vaquejador,PI


In [ ]:
geocode_incend = pd.merge(geocode_incend.rename(columns = {"address": "geo_address"}), incendios[["geo_address", "MUNICIPIO", "UF"]].drop_duplicates(subset=["geo_address"]), on=["geo_address"], how="left")

Tentativa de extração dos últimos registros que não tiveram geocode identificado pela API Google Maps.

In [ ]:
for i in tqdm(geocode_incend[geocode_incend["lat"].isnull()].index):
  try:
    result = gmaps.geocode(geocode_incend["MUNICIPIO"].loc[i] + geocode_incend["UF"].loc[i])
    geocode_incend["lat"].loc[i] = result[0]["geometry"]["location"]["lat"]
    geocode_incend["lng"].loc[i] = result[0]["geometry"]["location"]["lng"]
  except:
    pass

100%|██████████| 66/66 [00:06<00:00, 10.32it/s]


Merge e save do dataset.

In [ ]:
incendios = pd.merge(incendios, geocode_incend[["geo_address", "lat", "lng"]], how="left", on=["geo_address"])
incendios = incendios[["DATA/HORA INICIO FOGO", "LOCALIDADE", "MUNICIPIO", "UF", "TIPO ITEM", "PROVAVEL CAUSA", "lat", "lng"]].rename(columns = {"DATA/HORA INICIO FOGO": "data_inicio_fogo", "LOCALIDADE": "localidade", 
                                                                                                                           "MUNICIPIO": "mun", "UF": "uf", "TIPO ITEM": "tipo_item", "PROVAVEL CAUSA": "provavel_causa",
                                                                                                                            })
incendios = incendios.drop_duplicates()
incendios.to_csv("/content/drive/MyDrive/BNDES_premio_dados/dados_meio_ambiente/dados_cnpj/incendios_local.csv", index=False)

Sample do dataset pronto.

In [ ]:
incendios

,data_inicio_fogo,localidade,mun,uf,tipo_item,provavel_causa,lat,lng
0,28/08/2017 12:08:00,FLONA DE BRASÍLIA,BRASILIA,DF,ATIVIDADE AGROPECUÁRIA,QUEIMA DE LIXO,-15.7865,-48.0578
15,28/02/2019 11:02:00,PARNA E HISTÓRICO DO MONTE PASCOAL,PORTO SEGURO,BA,OUTRAS CAUSAS,VANDALISMO,-16.8927,-39.2167
39,05/04/2016 20:04:00,APA SANTO ANTÔNIO,SANTA CRUZ CABRALIA,BA,OUTRAS CAUSAS,INDETERMINADO,-16.1838,-38.9636
43,29/10/2019 10:10:00,PARQUE NATURAL SERRA DAS ALMAS,RIO DE CONTAS,BA,OUTRAS CAUSAS,INDETERMINADO,-13.3716,-41.8841
44,02/03/2016 08:03:00,VALE DO RIO BURANHÉM,PORTO SEGURO,BA,OUTRAS CAUSAS,INDETERMINADO,-16.4179,-39.5367
...,...,...,...,...,...,...,...,...
413576,NaN,PA SALUBRINHO / ANDARAI,ANDARAI,BA,NaN,NaN,-12.8006,-41.3271
413579,06/02/2019 17:02:00,TABALASCADA,CANTA,RR,NaN,NaN,2.68628,-60.6017
413587,04/10/2020 13:10:00,PARQUE DO XINGU,FELIZ NATAL,MT,ACIDENTE,CONFECÇÃO DE ACEIRO,-11.8625,-53.5278
413588,28/08/2017 13:08:00,PDS TERRA NOSSA / ALTAMIRA,ALTAMIRA,PA,ATIVIDADE AGROPECUÁRIA,RENOVAÇÃO DE PASTAGEM PLANTADA,-3.20311,-52.2045
